In [ ]:
import os
import subprocess
import numpy as np


In [ ]:
binf = '../ccv/bin/tld'

def_p = {
'init_radius': 12,
'win_radius' : 7,
'level' : 5,
'min_forward_backward_error' : 100,
'min_eigen' : 0.025,
'min_win' : 20,
'interval' : 3,
'shift' : 0.1,
'top_n' : 100,
'include_overlap' : 0.7,
'exclude_overlap' : 0.2,
'structs' : 40,
'features' : 18,
'validate_set' : 0.5,
'nnc_same' : 0.95,
'nnc_thres' : 0.65,
'nnc_verify' : 0.7,
'nnc_beyond' : 0.8,
'nnc_collect' : 0.5,
'bad_patches' : 100,
'new_deform' : 20,
'track_deform' : 10,
'new_deform_angle' : 20,
'track_deform_angle' : 10,
'new_deform_scale' : 0.02,
'track_deform_scale' : 0.02,
'new_deform_shift' : 0.02,
'track_deform_shift' : 0.02,
'tld_patch_size' : 10,
'tld_grid_sparsity' : 10}
new_p = {
'init_radius': 23,
'win_radius' : 7,
'level' : 5,
'min_forward_backward_error' : 124,
'min_eigen' : 0.02,
'min_win' : 23,
'interval' : 4,
'shift' : 0.07,
'top_n' : 133,
'include_overlap' : 0.70,
'exclude_overlap' : 0.26,
'structs' : 66,
'features' : 12,
'validate_set' : 0.41,
'nnc_same' : 0.71,
'nnc_thres' : 0.92,
'nnc_verify' : 0.76,
'nnc_beyond' : 0.74,
'nnc_collect' : 0.54,
'bad_patches' : 73,
'new_deform' : 15,
'track_deform' : 13,
'new_deform_angle' : 21,
'track_deform_angle' : 4,
'new_deform_scale' : 0.04,
'track_deform_scale' : 0.02,
'new_deform_shift' : 0.03,
'track_deform_shift' : 0.02,
'tld_patch_size' : 10,
'tld_grid_sparsity' : 11}
p_order = list(def_p.keys())
x0 = np.log([new_p[k] for k in p_order]) # 17370.702460478657

In [ ]:
def eval_points(x,name,pt):
    init_r = int(np.round(np.exp(x[0])))
    extra_s = ['{:.3f}'.format(_) for _ in np.exp(x[1:])]
    cmd_str = [str(_) for _ in [binf,'DAVIS/Videos/{}.mp4'.format(name),pt[1]-init_r,pt[0]-init_r,1+init_r*2,1+init_r*2]] + extra_s
    outp = subprocess.check_output(cmd_str)
    idx_to_loc = {}
    for row in outp.decode().split('\n'):
        if len(row) == 0:
            continue
        if '---' in row:
            pass
        else:
            loc = np.array([int(_) for _ in row.split()[1:3]]) + (np.array([int(_) for _ in row.split()[3:5]])-1)/2
        idx_to_loc[int(row[:5])] = np.array([loc[1],loc[0]])
    return idx_to_loc

In [ ]:
import json
import multiprocessing
cpu_count = multiprocessing.cpu_count()//2
pool = multiprocessing.Pool(cpu_count)

In [ ]:
def eval_cfg(x):
    try:
        errs = 0 

        for ef in os.listdir('joint_annotations/'):
            base,ext = os.path.splitext(ef)
            jf = os.path.join('joint_annotations',ef)
            with open(jf,'rt') as fp:
                data = json.load(fp)
            if '00000' not in data[0]['image_path'] :
                continue
            fold = data[0]['image_path'].split('/')[-2]
            if 'horsejump-high' not in fold:
                continue

            run_points = []
            for ji,j in enumerate(data[0]['joints']):
                if np.sum(j) == 0:
                    continue
                run_points.append([ji,j])
                if len(run_points) >= cpu_count:
                    break
            pret = pool.starmap(eval_points,[(x,fold,p[1]) for p in run_points])
            for elem in data:
                idx = int(elem['image_path'][-9:-4])
                for p,res in zip(run_points,pret):
                    t_pt = elem['joints'][p[0]]
                    errs += np.linalg.norm(res.get(idx,res[0]) - t_pt)
        return errs
    except KeyboardInterrupt:
        raise
    except:
        return 1e12
eval_cfg(x0)

In [ ]:
import cma
es = cma.CMAEvolutionStrategy(x0,0.1)
es.optimize(eval_cfg)

In [ ]:
es.best.x

In [ ]:
es.mean

In [ ]:
' '.join(['{:.3f}'.format(_) for _ in np.exp(es.best.x)])